## Dependencies



In [1]:
!pip install mftool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [2]:
from mftool import Mftool
import pandas as pd
import datetime

## Select Mutual Fund House and Scheme

In [3]:
mf = Mftool()

In [5]:
# get all supported amc details
amc_details = mf.get_all_amc_profiles(as_json=False)
# amc_details

In [6]:
df_amcs = pd.DataFrame(amc_details)
# df_amcs

In [7]:
# let's select a reputed fund house
df_amc = df_amcs[df_amcs["Name of the Mutual Fund"] == "PPFAS Mutual Fund"]
df_amc

,Name of the Mutual Fund,Date of set up of Mutual Fund,Name(s) of Sponsor,Name of Trustee Company,Name of Trustees,Name of Assest Management Co.,Date of Incorporation of AMC,Name(s) of Director,Name of Chairman,Name of Chief Executive Officer,...,Name(s) of Company Secretary,Name(s) of Fund Manager,Name of Compliance Officer,Name of Chief Bussiness Officer,Name of the Chief Investment Officer,Name(s) of the Chief Investment Officer - Dept,Name of Head of Operations,Name(s) of the Chief Investment Officer - Equity,Name of President,Name of Wholetime Director
31,PPFAS Mutual Fund,"October 10, 2012",Parag Parikh Financial Advisory Services Ltd.,PPFAS Trustee Company Private Limited,Bhagirat Merchant - Independent Director ...,PPFAS Asset Management Private Limited,"August 08, 2011",Neil Parag Parikh ...,NaN,Neil Parag Parikh,...,NaN,Raj Mehta ...,Priya Hariani,NaN,Rajeev Thakkar,NaN,Jignesh Desai ...,NaN,NaN,Rajeev Thakkar


In [8]:
# what is the fund house ID?
df_amc.columns.to_list()

['Name of the Mutual Fund',
 'Date of set up of Mutual Fund',
 'Name(s) of Sponsor',
 'Name of Trustee Company',
 'Name of Trustees',
 'Name of Assest Management Co.',
 'Date of Incorporation of AMC',
 'Name(s) of Director',
 'Name of Chairman',
 'Name of Chief Executive Officer',
 'Name of Managing Director',
 'Name of Compliance Officer & Company Secretary',
 'Name of Investor Service Officer',
 'Address of AMC',
 'Telephone Number',
 'Fax Number',
 'Website',
 'Email',
 'Name(s) of Auditors ',
 'Name(s) of Custodian ',
 'Name(s) of Registrar and Transfer Agent',
 'Name of Head Equity',
 'Name of Head-Fixed Income',
 'Name of Sales Head',
 'Name(s) of the Chief Operating Officer',
 'Name(s) of Company Secretary',
 'Name(s) of Fund Manager',
 'Name of Compliance Officer',
 'Name of Chief Bussiness Officer',
 'Name of the Chief Investment Officer',
 'Name(s) of the Chief Investment Officer - Dept',
 'Name of Head of Operations',
 'Name(s) of the Chief Investment Officer - Equity',
 'Na

In [9]:
# no ID here
# let's fetch funds now

df_schemes = pd.Series(mf.get_scheme_codes())
# df_schemes

In [10]:
# filter the funds from selected fund house
df_schemes[df_schemes.str.contains("Parag Parikh")]

143263    Parag Parikh Liquid Fund- Direct Plan- Daily R...
143269        Parag Parikh Liquid Fund- Direct Plan- Growth
143262    Parag Parikh Liquid Fund- Direct Plan- Monthly...
143265    Parag Parikh Liquid Fund- Direct Plan- Weekly ...
143264    Parag Parikh Liquid Fund- Regular Plan- Daily ...
143260       Parag Parikh Liquid Fund- Regular Plan- Growth
143261    Parag Parikh Liquid Fund- Regular Plan- Monthl...
143266    Parag Parikh Liquid Fund- Regular Plan- Weekly...
147481           Parag Parikh Tax Saver Fund- Direct Growth
147482          Parag Parikh Tax Saver Fund- Regular Growth
122639    Parag Parikh Flexi Cap Fund - Direct Plan - Gr...
122640    Parag Parikh Flexi Cap Fund - Regular Plan - G...
148958    Parag Parikh Conservative Hybrid Fund - Direct...
148961    Parag Parikh Conservative Hybrid Fund - Direct...
148959    Parag Parikh Conservative Hybrid Fund - Regula...
148960    Parag Parikh Conservative Hybrid Fund - Regula...
dtype: object

In [11]:
# let's select the fund category
df_fund = df_schemes[(df_schemes.str.contains("Parag Parikh")) & (df_schemes.str.contains("Flexi")) & (df_schemes.str.contains("Direct"))]
df_fund

122639    Parag Parikh Flexi Cap Fund - Direct Plan - Gr...
dtype: object

## Collect Historic Data & Make Ready To Access

In [56]:
df_nav = mf.get_scheme_historical_nav(df_fund.index.item(), as_Dataframe=True)
df_nav

,nav,dayChange
date,,
04-11-2022,50.96810,-0.3444
03-11-2022,51.31250,-0.3929
02-11-2022,51.70540,-0.2795
01-11-2022,51.98490,0.0385
31-10-2022,51.94640,0.4248
...,...,...
03-06-2013,10.05720,0.0418
31-05-2013,10.01540,-0.0173
30-05-2013,10.03270,0.0247


In [58]:
df_nav.dtypes

nav           object
dayChange    float64
dtype: object

In [59]:
# code in NAVAccessor below
# # cast nav to float64

# df_nav["nav"] = df_nav["nav"].astype("float64")
# df_nav.dtypes

In [60]:
# code in NAVAccessor below
# # cast index to datetime

# df_nav.index = pd.to_datetime(df_nav.index)
# df_nav.index.dtype

In [61]:
# check for saturday and sunday nav

today = datetime.datetime.today().date()
sunday_idx = (today.weekday() + 1) % 7

sun = today - datetime.timedelta(days=sunday_idx)
sun

datetime.date(2022, 10, 30)

In [62]:
try:
  df_nav.loc[str(sun)]
except:
  print("Data missing for Sunday. As expected!")

Data missing for Sunday. As expected!


In [69]:
# data will always be missing for holidays, and weekends
# on holidays, the nav stays the same as previous day's nav

# refer: https://pandas.pydata.org/pandas-docs/stable/development/extending.html

@pd.api.extensions.register_dataframe_accessor("safe_nav")
class NAVAccessor:
    def __init__(self, pandas_obj):
        self._validate(pandas_obj)
        pandas_obj = self._preprocess(pandas_obj)
        self.start_date = pandas_obj.index[-1]
        self._obj = pandas_obj

    @staticmethod
    def _validate(obj):
        # verify there is a column nav and a column dayChange,
        # and index must be datetime
        if "nav" not in obj.columns or "dayChange" not in obj.columns:
            raise AttributeError("Must have 'nav' and 'dayChange'.")

    @staticmethod
    def _preprocess(obj):
        # search, sort, type conversions, etc.
        # assumption: dataframe will always be sorted by date in descending order
        
        # todo: note: datetime index will be better as it is faster then object index
        # this will convert index's dtype to datetime.date object
        obj.index = pd.to_datetime(obj.index, format="%Y-%m-%d")

        obj.index = obj.index.date

        # other conversions
        obj["nav"] = obj["nav"].astype("float64")

        return obj

    # let's create a separate function which handles this
    def for_date(self, date):
      if date > self.start_date:
        # go back by a day till holidays are not over 
        while date not in self._obj.index:
          date = date - datetime.timedelta(days=1)
        
        # return the data of the day before holidays
        return self._obj.loc[date]

      # return the data of inception if data is from very past
      return self._obj.loc[self.start_date]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: registration of accessor <class '__main__.NAVAccessor'> under name 'safe_nav' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  


In [73]:
# verify index and format
df_nav.index

Index([2022-11-04, 2022-11-03, 2022-11-02, 2022-11-01, 2022-10-31, 2022-10-28,
       2022-10-27, 2022-10-25, 2022-10-21, 2022-10-20,
       ...
       2013-06-10, 2013-06-07, 2013-06-06, 2013-06-05, 2013-06-04, 2013-06-03,
       2013-05-31, 2013-05-30, 2013-05-29, 2013-05-28],
      dtype='object', length=2323)

In [74]:
df_nav.safe_nav._obj.index

Index([2022-11-04, 2022-11-03, 2022-11-02, 2022-11-01, 2022-10-31, 2022-10-28,
       2022-10-27, 2022-10-25, 2022-10-21, 2022-10-20,
       ...
       2013-06-10, 2013-06-07, 2013-06-06, 2013-06-05, 2013-06-04, 2013-06-03,
       2013-05-31, 2013-05-30, 2013-05-29, 2013-05-28],
      dtype='object', length=2323)

In [76]:
# check if it works
df_nav.safe_nav.for_date(sun)

nav          51.5216
dayChange    -0.5891
Name: 2022-10-28, dtype: float64

In [77]:
# shortcut
df_nav = df_nav.safe_nav
df_nav.for_date(sun)

nav          51.5216
dayChange    -0.5891
Name: 2022-10-28, dtype: float64

## Calculate Returns

### Absolute Returns

In [95]:
def one_day_absolute_return(df_nav):
  today_date = datetime.datetime.today().date()
  yesterday_date = (today_date - datetime.timedelta(days=1))
  day_before_yesterday_date = (today_date - datetime.timedelta(days=2))
  
  return df_nav.for_date(yesterday_date)["dayChange"]*100 / df_nav.for_date(day_before_yesterday_date)["nav"]

one_day_absolute_return(df_nav)

-0.6711814859926923

In [96]:
def one_year_absolute_return(df_nav):
  today_date = datetime.datetime.today().date()
  yesterday_date = (today_date - datetime.timedelta(days=1))
  one_year_before_yesterday_date = (today_date - datetime.timedelta(days=366))

  return (df_nav.for_date(yesterday_date)["nav"] - df_nav.for_date(one_year_before_yesterday_date)["nav"])*100 / df_nav.for_date(one_year_before_yesterday_date)["nav"]

one_year_absolute_return(df_nav)

-4.817920370508706

In [97]:
# generic function

def n_years_absolute_return(df_nav, n_years=7):
  today_date = datetime.datetime.today().date()
  yesterday_date = (today_date - datetime.timedelta(days=1))
  n_years_before_yesterday_date = (yesterday_date - datetime.timedelta(days=365*n_years))

  return (df_nav.for_date(yesterday_date)["nav"] - df_nav.for_date(n_years_before_yesterday_date)["nav"])*100 / df_nav.for_date(n_years_before_yesterday_date)["nav"]

In [99]:
for n_years in (1, 3, 5, 7):
  return_ = n_years_absolute_return(df_nav, n_years=n_years)
  print("Years:", n_years, "\t", "Abs. Return:", return_)

Years: 1 	 Abs. Returns: -4.817920370508706
Years: 3 	 Abs. Returns: 85.65517191136884
Years: 5 	 Abs. Returns: 119.47250570555055
Years: 7 	 Abs. Returns: 196.37096305255452


### Trailing Returns

**Understanding**

Trailing Return cancels the compounding effect from the absolute return.

If it is a year-on-year compounding, trailing return tells you the yearly simple interest rate that would have applied to compound and reach the final amount in the given time span.

For example, the absolute return for 7 years, which is 196.37% does not consider the time that you'll have to wait to earn this much profit.

Like if we want to compare the return of this mutual fund with a typical compound interest scheme, we want to know the yearly simple interest rate that applies. What if a compound interest scheme for 7 years would generate more than 196.37% absolute return? You'd never know.

Trailing return helps us with calculating the yearly rate of interest that you'd need to reach upto 196.37% in 7 years. Basically, it remove the timefactor from the absolute return and gets you the yearly interest rate.

In [ ]:
# formulation

# compound interest formula

# final_value = initial_value * (1 + (interest_rate / num_times_interest_applied_per_period) ^ (num_times_interest_applied_per_period*num_periods))

# for 5 year compounding and interest applied yearly
# num_times_interest_applied_per_period = 1, num_periods=5

# for 5 year compounding and interest applied mothly
# num_times_interest_applied_per_period = 12, num_periods=5


# deriving the formula of trailing returns from above, we get

# trailing_return = ((final_value / initial_value) ^ (1 / num_times_interest_applied_per_period*num_periods) - 1) * num_times_interest_applied_per_period)

In [104]:
# for most of the compounding schemes, num_times_interest_applied_per_period = 1

def n_years_trailing_return(df_nav, n_years):
  # redundancy count: 2
  today_date = datetime.datetime.today().date()
  yesterday_date = (today_date - datetime.timedelta(days=1))
  n_years_before_yesterday_date = (yesterday_date - datetime.timedelta(days=365*n_years))

  final_value = df_nav.for_date(yesterday_date)["nav"]
  initial_value = df_nav.for_date(n_years_before_yesterday_date)["nav"]

  return (((final_value / initial_value) ** (1 / n_years)) - 1)*100

In [105]:
for n_years in (1, 3, 5, 7):
  return_ = n_years_trailing_return(df_nav, n_years=n_years)
  print("Years:", n_years, "\t", "Trl. Return:", return_)

Years: 1 	 Trl. Return: -4.817920370508711
Years: 3 	 Trl. Return: 22.904848946865954
Years: 5 	 Trl. Return: 17.024292561361843
Years: 7 	 Trl. Return: 16.789847958013524


**Observations:**
1. The fund has performed significantly well over the past 3 years, however, it's performance dropped last year. Basically, the fund performed very well from November 2019 to November 2021, and it's performance dropped after November 2021 till November 2022.
2. We can confirm the same by looking at absolute returns. Out of 119.47% over 5 years, 85.65% alone was generate in the last 3 years.
3. The fund is not able to maintain it's yearly interest rate i.e. trailing return value in the last year. If you had to invest in this fund, ignoring the last year, you could expect an average annual return of 16% or more in this fund, given that you stay invested for more than 3 years.

**NOTE:** For 1 year, trailing return = absolute return as there is no compounding effect